# Solving Business Problems with AI

## Objective
Develop a proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalog information and stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

## Instructions

- All requirements have to be implemented with the use of LLMs to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Because solving this assessment requires using advanced LLM capabilities, we provide you with a temporary OpenAI API key granting access to GPT-4o. You can use this key or your own, but please note that the provided key has a token quota, so use it wisely. We have carefully adjusted the limits to ensure they are sufficient for you to complete the task.
- Address the requirements in the order listed. Please review the requirements ahead to have a general implementation plan in place before you start.
- Your deliverables should include the code developed within this notebook, and a spreadsheet containing results across separate sheets. Comments detailing your thought process are appreciated.
- You are allowed to use additional libraries (langchain, etc.) in order to make solution more concise.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, create a new order line with the status **created**. If the order cannot be fully fulfilled, create a line with the status **out of stock** with quantity matching requested value. After placing the order, update the stock to accurately reflect the current inventory levels. Create a record for each product requested in the email.

    **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate and save response emails based on order processing results. Depending on the order status email should inform customer that their order was processed fully or partially, or could not be fulfilled. If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock. Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

    **Output**: Populate the **order-response** sheet with columns: email ID, response.

### 3. Handle product inquiry

Customers may ask general open questions.
- Respond to product inquiries using the information from the product catalog.
- Ensure that solution you provide scales well to handle the full catalog of over 100,000 products. This means that solutions which include raw product catalog in the prompt will be rated lower, be catious on token limits!

**Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria

- **Utilization of AI Tools**: Effectiveness in leveraging AI technologies provided (e.g., OpenAI API) to fulfill the task requirements. Employ your knowledge of various AI driven development techniques depending on the task.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational.
- **Code Quality and Clarity**: Code should be well-organized with clear logic and structure.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the spreadsheet. Double check before submitting!

We look forward to seeing your solution and how you approach solving real-world problems using AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package.
%pip install openai==0.28

In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [ ]:
import pandas as pd
import gspread

import openai


# OpenAI API Key
openai.api_key = 'sk-PuMkpn6P0MEtXAbYDa2qXIXikwqXd0FqKKTvyqYXSOT3BlbkFJESv9YI2iVs91OpaRVgnSCxb2nbn8OnevPofO1TG3YA'

import pandas as pd
import openai

# Function to classify emails
def classify_email(email_body):
    prompt = f"""
    Classify the following email as either a "product inquiry" or an "order request". The classification should accurately reflect the intent of the email.

    Email: {email_body}

    Classification:"""

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=10,
        n=1,
        stop=None,
        temperature=0
    )

    classification = response.choices[0].text.strip().lower()
    return classification


# Classify each email
emails_df['category'] = emails_df['message'].apply(classify_email)

# Create a new DataFrame with the required columns
classification_df = emails_df[['email ID', 'category']]

# Save this DataFrame to a new Excel file
classification_df.to_excel('email_classification.xlsx', index=False)

print("Email classification completed and saved to email_classification.xlsx.")



# Task 2. Process order requests

In [ ]:
def extract_product_and_quantity_from_message(email_body):
    prompt = f"""
    Extract the product name or description and the quantity requested from the following email message.
    Please provide the product and quantity in a JSON format with keys "product" and "quantity".

    Email: {email_body}

    Output:"""

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=60,
        n=1,
        stop=None,
        temperature=0
    )

    try:
        extracted_data = eval(response.choices[0].text.strip())
        product = extracted_data.get("product", "").strip()
        quantity = int(extracted_data.get("quantity", "").strip())
    except:
        product = ""
        quantity = 0

    return product, quantity

# Function to match the extracted product with the product description and check availability
def match_product_and_check_availability(extracted_product, requested_quantity, products_df):
    prompt = f"Given the following product descriptions, find the closest match for the product '{extracted_product}' and return the corresponding product ID:\n\n"

    descriptions = "\n".join([f"Product ID: {row['product ID']}, Description: {row['detailed description']}" for index, row in products_df.iterrows()])

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt + descriptions,
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0
    )

    matched_product_id = response.choices[0].text.strip().split(":")[-1].strip()  # Assuming the model returns something like "Product ID: XYZ"

    # Check the availability in stock
    product_row = products_df[products_df['product ID'] == matched_product_id]
    if not product_row.empty:
        available_stock = product_row['stock amount'].values[0]
        status = "available" if available_stock >= requested_quantity else "out of stock"
    else:
        status = "out of stock"

    return matched_product_id, status

    extracted_product_status = []

# Loop over each email classified as an "order request"
for index, row in emails_df.iterrows():
    email_id = row['email_id']
    email_body = row['message']

    # Step 1: Extract the product and quantity from the email message
    extracted_product, requested_quantity = extract_product_and_quantity_from_message(email_body)

    # Step 2: Match the extracted product with the product descriptions and check availability
    product_id, availability_status = match_product_and_check_availability(extracted_product, requested_quantity, products_df)

    # Store the email ID, product ID, quantity, and availability status
    extracted_product_status.append([email_id, product_id, extracted_product, requested_quantity, availability_status])

# Convert results to DataFrame
order_status = pd.DataFrame(extracted_product_status, columns=['email ID', 'product ID', 'extracted product', 'requested quantity', 'status'])

# Display the first few results
print(order_status.head())

# Optionally, save the results to a new Excel file
order_status.to_excel('extracted_products_with_status.xlsx', index=False)

2.2Updating Stock

In [ ]:
order_status = pd.read_excel('extracted_products_with_status.xlsx')

# Step 1: Filter rows with status "available"
available_orders_df = order_status[order_status['status'] == 'available']

# Step 2: Update the stock in products_df
for _, row in available_orders_df.iterrows():
    product_id = row['product ID']
    requested_quantity = row['requested quantity']

    # Find the corresponding row in products_df
    product_row_index = products_df[products_df['product ID'] == product_id].index[0]

    # Subtract the requested quantity from the stock
    products_df.at[product_row_index, 'stock amount'] -= requested_quantity

# Step 3: Save the updated products_df back to a CSV or Excel file
products_df.to_csv(f"updated_products_{document_id}.csv", index=False)

# Optionally, save as Excel if needed
products_df.to_excel(f"updated_products_{document_id}.xlsx", index=False)

print("Stock levels have been updated based on available orders and saved.")

2.3 Generating emails.

In [ ]:
import pandas as pd

# Function to generate email responses based on order status
def generate_response_email(email_id, product_name, quantity, status):
    if status == "available":
        response = f"""
        Dear Customer,

        We are pleased to inform you that your order for {quantity} units of {product_name} has been successfully processed. Your items are now being prepared for shipment and will be dispatched shortly.

        Thank you for shopping with us!

        Best regards,
        Your Company Name
        """
    elif status == "out of stock":
        response = f"""
        Dear Customer,

        Unfortunately, we regret to inform you that your order for {quantity} units of {product_name} could not be fulfilled due to insufficient stock.

        We apologize for the inconvenience this may have caused. If you would like to wait for a restock or choose an alternative product, please contact us at your earliest convenience.

        Thank you for your understanding.

        Best regards,
        Your Company Name
        """
    else:
        response = f"""
        Dear Customer,

        There was an issue processing your order for {quantity} units of {product_name}. Please contact our support team for further assistance.

        We apologize for any inconvenience this may have caused.

        Best regards,
        Your Company Name
        """

    return response

# Load the order_status Excel file
order_status_df = pd.read_excel('order_status.xlsx')

# Initialize a list to store the responses
order_responses = []

# Generate responses for each order based on status
for index, row in order_status_df.iterrows():
    email_id = row['email ID']
    product_name = row['extracted product']
    quantity = row['requested quantity']
    status = row['status']

    # Generate the email response
    response = generate_response_email(email_id, product_name, quantity, status)

    # Store the email ID and response
    order_responses.append([email_id, response])

# Convert responses to a DataFrame
order_response_df = pd.DataFrame(order_responses, columns=['email ID', 'response'])

# Save the responses to a new Excel file or sheet
order_response_df.to_excel('order_response.xlsx', index=False)

print("Order responses have been generated and saved to order_response.xlsx.")


# Task 3. Handle product inquiry

In [ ]:

# Load the necessary data
email_classification_df = pd.read_excel('email_classification.xlsx')
emails_df = pd.read_excel('emails_df.xlsx')
products_df = pd.read_excel('products_df.xlsx')

# Function to generate product inquiry response
def generate_product_inquiry_response(message, products_df):
    # Extract product-related information from the message
    prompt = f"""
    Extract the product-related inquiry from the following message and generate a response using the product descriptions available. If the product is mentioned, provide a detailed response including the product's description from the catalog:

    Message: {message}

    Available Product Descriptions:
    """

    descriptions = "\n".join([f"Product: {row['name']}, Description: {row['description']}" for index, row in products_df.iterrows()])

    full_prompt = prompt + descriptions

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=full_prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5
    )

    response_text = response.choices[0].text.strip()
    return response_text

# Initialize a list to store the responses
product_inquiry_responses = []

# Step 1: Identify product inquiries in the email_classification_df
product_inquiry_df = email_classification_df[email_classification_df['category'] == 'product inquiry']

# Step 2: Match email IDs and extract corresponding messages
for _, row in product_inquiry_df.iterrows():
    email_id = row['email ID']

    # Find the corresponding message in emails_df
    message = emails_df[emails_df['email ID'] == email_id]['message'].values[0]

    # Step 3: Generate the email response using the product descriptions
    response = generate_product_inquiry_response(message, products_df)

    # Store the email ID and the generated response
    product_inquiry_responses.append([email_id, response])

# Convert responses to a DataFrame
product_inquiry_response_df = pd.DataFrame(product_inquiry_responses, columns=['email ID', 'response'])

# Save the responses to a new Excel file or sheet
product_inquiry_response_df.to_excel('product_inquiry_responses.xlsx', index=False)

print("Product inquiry responses have been generated and saved to product_inquiry_responses.xlsx.")
